In [1]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import Ridge, HuberRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings("ignore")

## Functions

In [2]:
def buildNN (data, num_hidden_layers = 4, hidden_nodes = 64, act = 'relu', do = 0, regularizer = True,
                loss_function = 'mean_absolute_error'):
    
    if regularizer:
        reg = tf.keras.regularizers.l2(l=0.01)
    else:
        reg = None
        
    NN_model = Sequential()
    
    # The Input Layer :
    input_layer= Input(shape=(data.shape[1],))
    NN_model.add(input_layer)

    NN_model.add(Dense(hidden_nodes, kernel_initializer='normal',activation=act, kernel_regularizer = reg))
    # The Hidden Layers :
    for i in range(num_hidden_layers-1):
        if (do>0):
            NN_model.add(Dropout(do))
        NN_model.add(Dense(hidden_nodes, kernel_initializer='normal',activation=act, kernel_regularizer = reg))
        

    # The Output Layer :
    NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

    # Compile the network :
    NN_model.compile(loss= loss_function, optimizer='adam', metrics=['mean_absolute_error', 'mean_squared_error'])
    
    return NN_model

In [3]:
def build_autoencoder(data, num_hidden_layers = 4, num_nodes = 32, act = 'relu'):

    model=Sequential()
    
    input_layer= Input(shape=(data.shape[1],))
    model.add(input_layer)
    
    for i in range(num_hidden_layers):
        model.add(Dense(units = num_nodes, activation=act))

            
    model.add(Dense(units=data.shape[1], activation=act))
    
    model.compile(optimizer='adam', loss= 'mean_squared_error', metrics=['mse']) 
    
    return model

In [4]:
def evaluateModel (model, X, y):
    y_pred = model.predict(X)
    
    mse = mean_squared_error(y,y_pred)
    r2 = r2_score(y,y_pred)
    mae = mean_absolute_error(y,y_pred)
    
    return mse, r2, mae

## Data

In [5]:
data = pd.read_csv('input/processed_data_nyc.csv', index_col = 0)
data.head()

,latitude,longitude,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,all_year_avail,low_avail,...,neighbourhood_Williamsburg,neighbourhood_Willowbrook,neighbourhood_Windsor Terrace,neighbourhood_Woodhaven,neighbourhood_Woodlawn,neighbourhood_Woodrow,neighbourhood_Woodside,room_type_Entire home/apt,room_type_Private room,room_type_Shared room
0,40.64749,-73.97237,5.010635,1,9,2762,0.21,6,True,False,...,0,0,0,0,0,0,0,0,1,0
1,40.75362,-73.98377,5.420535,1,45,2976,0.38,2,True,False,...,0,0,0,0,0,0,0,1,0,0
2,40.80902,-73.94190,5.017280,3,0,0,0.00,1,True,False,...,0,0,0,0,0,0,0,0,1,0
3,40.68514,-73.95976,4.499810,1,270,3021,4.64,1,False,False,...,0,0,0,0,0,0,0,1,0,0
4,40.79851,-73.94399,4.394449,10,9,2793,0.10,1,False,True,...,0,0,0,0,0,0,0,1,0,0


In [6]:
y = data.price
data = data.drop(['price'], axis=1)

X = np.asarray(data).astype(np.float32)
y = np.asarray(y).ravel()

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Training Dataset: {}".format(X_train.shape))
print("Testing Dataset: {}".format(X_test.shape))

Training Dataset: (39014, 239)
Testing Dataset: (9754, 239)


In [8]:
scaler = preprocessing.RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

## Ensemble Architecture

The ensemble architecture will include all three regressor models (huber, rf, and ridge regressors) and a neural network; all trained on original processed data as well as new representation of the data retreived from the autoencoder. All 8 models (4 with and 4 without autoencoder) will be connected to a fusion layer which will be used as training data for the final neural network.

In [15]:
EPOCHS = 500
BATCH_SIZE = 128

# Models
sae = build_autoencoder(X_train, num_hidden_layers = 4, num_nodes = 32, act = 'relu')
rf = RandomForestRegressor(n_estimators=50)
ridge = Ridge(alpha=5)
huber = HuberRegressor(alpha=10, epsilon=3)
ann_inner = buildNN (data, num_hidden_layers = 3, hidden_nodes = 128, act = 'relu', do = 0, regularizer = True,
                loss_function = 'mean_squared_error')



earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      min_delta=0.0001, patience=10,
                                                      verbose=0, mode='auto')

# Get new representations
sae.fit(X_train, X_train, epochs = EPOCHS,  batch_size = BATCH_SIZE, validation_split = 0.2,
        callbacks = [earlystop_callback])
X_train_new = sae.predict(X_train)
X_test_new = sae.predict(X_test)

# Models on original representation
rf.fit(X_train, y_train)
ridge.fit(X_train, y_train)
huber.fit(X_train, y_train)
ann_inner.fit(X_train, y_train, epochs = EPOCHS, batch_size = BATCH_SIZE,
            validation_split = 0.2)

# Get original model outputs
rf_out = rf.predict(X_train)
ridge_out = ridge.predict(X_train)
huber_out = huber.predict(X_train)
ann_inner_out = ann_inner.predict(X_train)

rf_out_test = rf.predict(X_test)
ridge_out_test = ridge.predict(X_test)
huber_out_test = huber.predict(X_test)
ann_inner_out_test = ann_inner.predict(X_test)

# Models on new representation
rf.fit(X_train_new, y_train)
ridge.fit(X_train_new, y_train)
huber.fit(X_train_new, y_train)
ann_inner.fit(X_train_new, y_train, epochs = EPOCHS, batch_size = BATCH_SIZE,
            validation_split = 0.2, callbacks = [earlystop_callback])

# Get new representation model outputs
rf_out2 = rf.predict(X_train_new)
ridge_out2 = ridge.predict(X_train_new)
huber_out2 = huber.predict(X_train_new)
ann_inner_out2 = ann_inner.predict(X_train_new)

rf_out_test2 = rf.predict(X_test_new)
ridge_out_test2 = ridge.predict(X_test_new)
huber_out_test2 = huber.predict(X_test_new)
ann_inner_out_test2 = ann_inner.predict(X_test_new)

# Reshape
rf_out = rf_out.reshape(X_train.shape[0], 1)
ridge_out = ridge_out.reshape(X_train.shape[0], 1)
huber_out = huber_out.reshape(X_train.shape[0], 1)
rf_out2 = rf_out2.reshape(X_train.shape[0], 1)
ridge_out2 = ridge_out2.reshape(X_train.shape[0], 1)
huber_out2 = huber_out2.reshape(X_train.shape[0], 1)

rf_out_test = rf_out_test.reshape(X_test.shape[0], 1)
ridge_out_test = ridge_out_test.reshape(X_test.shape[0], 1)
huber_out_test = huber_out_test.reshape(X_test.shape[0], 1)
rf_out_test2 = rf_out_test2.reshape(X_test.shape[0], 1)
ridge_out_test2 = ridge_out_test2.reshape(X_test.shape[0], 1)
huber_out_test2 = huber_out_test2.reshape(X_test.shape[0], 1)


# Creating fusion vectors
fused_train = (rf_out, ridge_out, huber_out, ann_inner_out, rf_out2, ridge_out2, huber_out2, ann_inner_out2)
fused_train = np.concatenate(fused_train, axis=1)
fused_test = (rf_out_test, ridge_out_test, huber_out_test, ann_inner_out_test,
              rf_out_test2, ridge_out_test2, huber_out_test2, ann_inner_out_test2)
fused_test = np.concatenate(fused_test, axis=1)

# Outer Neural Net
# ann_outer = buildNN (fused_train, num_hidden_layers = 3, hidden_nodes = 256, act = 'relu', do = 0, regularizer = True,
#                 loss_function = 'mean_absolute_error')

# ann_outer.fit(fused_train, y_train, epochs = EPOCHS, batch_size = BATCH_SIZE,
#             validation_split = 0.2, callbacks = [earlystop_callback])
model_outer = RandomForestRegressor(n_estimators=50)
model_outer.fit(fused_train, y_train)

print("Training Complete")

Train on 31211 samples, validate on 7803 samples
Epoch 1/500
31211/31211 [==============================] - 1s 30us/sample - loss: 1.0583 - mse: 1.0583 - val_loss: 0.1459 - val_mse: 0.1459
Epoch 2/500
31211/31211 [==============================] - 1s 17us/sample - loss: 0.1327 - mse: 0.1327 - val_loss: 0.1421 - val_mse: 0.1421
Epoch 3/500
31211/31211 [==============================] - 1s 19us/sample - loss: 0.1311 - mse: 0.1311 - val_loss: 0.1412 - val_mse: 0.1412
Epoch 4/500
31211/31211 [==============================] - 1s 17us/sample - loss: 0.1306 - mse: 0.1306 - val_loss: 0.1409 - val_mse: 0.1409
Epoch 5/500
31211/31211 [==============================] - 1s 18us/sample - loss: 0.1303 - mse: 0.1303 - val_loss: 0.1407 - val_mse: 0.1407
Epoch 6/500
31211/31211 [==============================] - 1s 17us/sample - loss: 0.1303 - mse: 0.1303 - val_loss: 0.1407 - val_mse: 0.1407
Epoch 7/500
31211/31211 [==============================] - 1s 18us/sample - loss: 0.1305 - mse: 0.1305 - val_lo

Epoch 20/500
31211/31211 [==============================] - 1s 18us/sample - loss: 0.2353 - mean_absolute_error: 0.3335 - mean_squared_error: 0.2045 - val_loss: 0.2316 - val_mean_absolute_error: 0.3285 - val_mean_squared_error: 0.2012
Epoch 21/500
31211/31211 [==============================] - 1s 18us/sample - loss: 0.2355 - mean_absolute_error: 0.3334 - mean_squared_error: 0.2049 - val_loss: 0.2300 - val_mean_absolute_error: 0.3271 - val_mean_squared_error: 0.2007
Epoch 22/500
31211/31211 [==============================] - 1s 19us/sample - loss: 0.2333 - mean_absolute_error: 0.3323 - mean_squared_error: 0.2032 - val_loss: 0.2328 - val_mean_absolute_error: 0.3255 - val_mean_squared_error: 0.2033
Epoch 23/500
31211/31211 [==============================] - 1s 18us/sample - loss: 0.2330 - mean_absolute_error: 0.3320 - mean_squared_error: 0.2033 - val_loss: 0.2340 - val_mean_absolute_error: 0.3362 - val_mean_squared_error: 0.2042
Epoch 24/500
31211/31211 [==============================] - 

Epoch 55/500
31211/31211 [==============================] - 1s 18us/sample - loss: 0.2187 - mean_absolute_error: 0.3248 - mean_squared_error: 0.1952 - val_loss: 0.2200 - val_mean_absolute_error: 0.3212 - val_mean_squared_error: 0.1968
Epoch 56/500
31211/31211 [==============================] - 1s 19us/sample - loss: 0.2203 - mean_absolute_error: 0.3268 - mean_squared_error: 0.1970 - val_loss: 0.2222 - val_mean_absolute_error: 0.3245 - val_mean_squared_error: 0.1990
Epoch 57/500
31211/31211 [==============================] - 1s 17us/sample - loss: 0.2186 - mean_absolute_error: 0.3250 - mean_squared_error: 0.1955 - val_loss: 0.2179 - val_mean_absolute_error: 0.3228 - val_mean_squared_error: 0.1949
Epoch 58/500
31211/31211 [==============================] - 1s 19us/sample - loss: 0.2192 - mean_absolute_error: 0.3261 - mean_squared_error: 0.1961 - val_loss: 0.2182 - val_mean_absolute_error: 0.3243 - val_mean_squared_error: 0.1958
Epoch 59/500
31211/31211 [==============================] - 

Epoch 90/500
31211/31211 [==============================] - 1s 18us/sample - loss: 0.2131 - mean_absolute_error: 0.3230 - mean_squared_error: 0.1934 - val_loss: 0.2146 - val_mean_absolute_error: 0.3233 - val_mean_squared_error: 0.1947
Epoch 91/500
31211/31211 [==============================] - 1s 17us/sample - loss: 0.2119 - mean_absolute_error: 0.3226 - mean_squared_error: 0.1925 - val_loss: 0.2111 - val_mean_absolute_error: 0.3249 - val_mean_squared_error: 0.1918
Epoch 92/500
31211/31211 [==============================] - 1s 18us/sample - loss: 0.2121 - mean_absolute_error: 0.3225 - mean_squared_error: 0.1927 - val_loss: 0.2158 - val_mean_absolute_error: 0.3354 - val_mean_squared_error: 0.1966
Epoch 93/500
31211/31211 [==============================] - 1s 17us/sample - loss: 0.2119 - mean_absolute_error: 0.3226 - mean_squared_error: 0.1925 - val_loss: 0.2124 - val_mean_absolute_error: 0.3239 - val_mean_squared_error: 0.1932
Epoch 94/500
31211/31211 [==============================] - 

Epoch 125/500
31211/31211 [==============================] - 1s 17us/sample - loss: 0.2106 - mean_absolute_error: 0.3235 - mean_squared_error: 0.1934 - val_loss: 0.2076 - val_mean_absolute_error: 0.3207 - val_mean_squared_error: 0.1903
Epoch 126/500
31211/31211 [==============================] - 1s 18us/sample - loss: 0.2097 - mean_absolute_error: 0.3221 - mean_squared_error: 0.1925 - val_loss: 0.2094 - val_mean_absolute_error: 0.3207 - val_mean_squared_error: 0.1921
Epoch 127/500
31211/31211 [==============================] - 1s 18us/sample - loss: 0.2095 - mean_absolute_error: 0.3222 - mean_squared_error: 0.1923 - val_loss: 0.2074 - val_mean_absolute_error: 0.3212 - val_mean_squared_error: 0.1902
Epoch 128/500
31211/31211 [==============================] - 1s 18us/sample - loss: 0.2104 - mean_absolute_error: 0.3227 - mean_squared_error: 0.1932 - val_loss: 0.2099 - val_mean_absolute_error: 0.3202 - val_mean_squared_error: 0.1929
Epoch 129/500
31211/31211 [=============================

...

Epoch 474/500
31211/31211 [==============================] - 1s 18us/sample - loss: 0.2029 - mean_absolute_error: 0.3206 - mean_squared_error: 0.1909 - val_loss: 0.2051 - val_mean_absolute_error: 0.3238 - val_mean_squared_error: 0.1933
Epoch 475/500
31211/31211 [==============================] - 1s 17us/sample - loss: 0.2023 - mean_absolute_error: 0.3196 - mean_squared_error: 0.1902 - val_loss: 0.2132 - val_mean_absolute_error: 0.3198 - val_mean_squared_error: 0.2014
Epoch 476/500
31211/31211 [==============================] - 1s 17us/sample - loss: 0.2028 - mean_absolute_error: 0.3210 - mean_squared_error: 0.1909 - val_loss: 0.2046 - val_mean_absolute_error: 0.3163 - val_mean_squared_error: 0.1928
Epoch 477/500
31211/31211 [==============================] - 1s 17us/sample - loss: 0.2032 - mean_absolute_error: 0.3209 - mean_squared_error: 0.1912 - val_loss: 0.2015 - val_mean_absolute_error: 0.3176 - val_mean_squared_error: 0.1895
Epoch 478/500
31211/31211 [=============================

31211/31211 [==============================] - 1s 20us/sample - loss: 0.2563 - mean_absolute_error: 0.3616 - mean_squared_error: 0.2479 - val_loss: 0.2585 - val_mean_absolute_error: 0.3644 - val_mean_squared_error: 0.2502
Epoch 9/500
31211/31211 [==============================] - 1s 18us/sample - loss: 0.2564 - mean_absolute_error: 0.3623 - mean_squared_error: 0.2480 - val_loss: 0.2577 - val_mean_absolute_error: 0.3614 - val_mean_squared_error: 0.2494
Epoch 10/500
31211/31211 [==============================] - 1s 17us/sample - loss: 0.2557 - mean_absolute_error: 0.3620 - mean_squared_error: 0.2473 - val_loss: 0.2579 - val_mean_absolute_error: 0.3652 - val_mean_squared_error: 0.2495
Epoch 11/500
31211/31211 [==============================] - 1s 18us/sample - loss: 0.2564 - mean_absolute_error: 0.3623 - mean_squared_error: 0.2480 - val_loss: 0.2566 - val_mean_absolute_error: 0.3598 - val_mean_squared_error: 0.2481
Epoch 12/500
31211/31211 [==============================] - 1s 21us/sample

Epoch 28/500
31211/31211 [==============================] - 1s 40us/sample - loss: 0.1168 - mean_absolute_error: 0.0835 - mean_squared_error: 0.0123 - val_loss: 0.1076 - val_mean_absolute_error: 0.0745 - val_mean_squared_error: 0.0102
Epoch 29/500
31211/31211 [==============================] - 1s 29us/sample - loss: 0.1170 - mean_absolute_error: 0.0841 - mean_squared_error: 0.0125 - val_loss: 0.1279 - val_mean_absolute_error: 0.0954 - val_mean_squared_error: 0.0150
Epoch 30/500
31211/31211 [==============================] - 1s 35us/sample - loss: 0.1123 - mean_absolute_error: 0.0798 - mean_squared_error: 0.0116 - val_loss: 0.1169 - val_mean_absolute_error: 0.0846 - val_mean_squared_error: 0.0121
Epoch 31/500
31211/31211 [==============================] - 4s 114us/sample - loss: 0.1122 - mean_absolute_error: 0.0801 - mean_squared_error: 0.0116 - val_loss: 0.1074 - val_mean_absolute_error: 0.0757 - val_mean_squared_error: 0.0104
Epoch 32/500
31211/31211 [==============================] -

Epoch 63/500
31211/31211 [==============================] - 2s 50us/sample - loss: 0.1005 - mean_absolute_error: 0.0768 - mean_squared_error: 0.0108 - val_loss: 0.0970 - val_mean_absolute_error: 0.0734 - val_mean_squared_error: 0.0100
Epoch 64/500
31211/31211 [==============================] - 2s 49us/sample - loss: 0.1044 - mean_absolute_error: 0.0809 - mean_squared_error: 0.0118 - val_loss: 0.1008 - val_mean_absolute_error: 0.0775 - val_mean_squared_error: 0.0107
Epoch 65/500
31211/31211 [==============================] - 2s 49us/sample - loss: 0.1037 - mean_absolute_error: 0.0803 - mean_squared_error: 0.0116 - val_loss: 0.0975 - val_mean_absolute_error: 0.0741 - val_mean_squared_error: 0.0102
Epoch 66/500
31211/31211 [==============================] - 1s 45us/sample - loss: 0.1001 - mean_absolute_error: 0.0769 - mean_squared_error: 0.0108 - val_loss: 0.0976 - val_mean_absolute_error: 0.0744 - val_mean_squared_error: 0.0101
Epoch 67/500
31211/31211 [==============================] - 

Epoch 98/500
31211/31211 [==============================] - 1s 45us/sample - loss: 0.0977 - mean_absolute_error: 0.0781 - mean_squared_error: 0.0111 - val_loss: 0.1075 - val_mean_absolute_error: 0.0880 - val_mean_squared_error: 0.0129
Epoch 99/500
31211/31211 [==============================] - 1s 43us/sample - loss: 0.1001 - mean_absolute_error: 0.0805 - mean_squared_error: 0.0116 - val_loss: 0.0927 - val_mean_absolute_error: 0.0732 - val_mean_squared_error: 0.0099
Epoch 100/500
31211/31211 [==============================] - 1s 42us/sample - loss: 0.0994 - mean_absolute_error: 0.0799 - mean_squared_error: 0.0114 - val_loss: 0.0940 - val_mean_absolute_error: 0.0745 - val_mean_squared_error: 0.0103
Epoch 101/500
31211/31211 [==============================] - 1s 40us/sample - loss: 0.0963 - mean_absolute_error: 0.0769 - mean_squared_error: 0.0109 - val_loss: 0.0927 - val_mean_absolute_error: 0.0733 - val_mean_squared_error: 0.0100
Epoch 102/500
31211/31211 [==============================]

In [18]:
model_outer = RandomForestRegressor(n_estimators=50)
model_outer.fit(fused_train, y_train)

RandomForestRegressor(n_estimators=50)

In [19]:
mse, r2, mae = evaluateModel(model_outer, fused_test, y_test)
print('MSE = {}'.format(mse))
print('R2 = {}'.format(r2))
print("MAE = {}".format(mae))

MSE = 0.18975169376407916
R2 = 0.5834976192563441
MAE = 0.31602044563900533


In [ ]:
a1 = np.arange(10).reshape(10,1)
a2 = np.arange(10).reshape(10,1)

In [ ]:
a3 = np.concatenate((a1, a2), axis = 1)

In [ ]:
X_train.shape[0]